In [54]:
import re
from urllib.parse import urlparse, parse_qs, unquote_plus
def normalize(value):
    if not value:
        return ""
    for _ in range(3):  # recursive decoding
        value = unquote_plus(value)
    value = value.lower()
    value = re.sub(r"\s+", " ", value)
    return value.strip()


In [55]:
#user input
def extract_user_input(log_line):
    """
    Extract ONLY attacker-controlled data:
    - Query parameter values
    - POST body parameters
    """
    inputs = []

    parts = log_line.split(",")

    # Extract URL query parameters
    for p in parts:
        if p.startswith("http"):
            parsed = urlparse(p)
            qs = parse_qs(parsed.query)
            for values in qs.values():
                inputs.extend(values)

    # Extract POST body (simple & safe heuristic)
    if "ip=" in log_line or "submit=" in log_line:
        body = parts[-1]
        inputs.append(body)

    return inputs


In [57]:

#sql

SQLI_SIGNS = [
    " or 1=1",
    " or '1'='1",
    "union select",
    "select ",
    "information_schema",
    "benchmark(",
    "sleep(",
    "--"
]

def detect_sqli(inputs):
    for data in inputs:
        d = normalize(data)
        for s in SQLI_SIGNS:
            if s in d:
                return True
    return False


In [58]:
#cmd
CMD_SIGNS = [";", "&&", "||", "|", "`", "$("]

def detect_cmd(inputs):
    for data in inputs:
        d = normalize(data)
        for s in CMD_SIGNS:
            if s in d:
                return True
    return False


In [59]:
#path
PATH_SIGNS = [
    "../",
    "..\\",
    "php://",
    "file://",
    "data://",
    "/etc/passwd"
]

def detect_path(inputs):
    for data in inputs:
        d = normalize(data)
        for s in PATH_SIGNS:
            if s in d:
                return True
    return False


In [60]:
#XSS
XSS_SIGNS = [
    "<script",
    "</script>",
    "javascript:",
    "onerror=",
    "onload=",
    "<img",
    "<svg",
    "<iframe"
]

def detect_xss(inputs):
    for data in inputs:
        d = normalize(data)
        for s in XSS_SIGNS:
            if s in d:
                return True
    return False


In [61]:
#file uplad
DANGEROUS_UPLOAD_EXT = [".php", ".py", ".exe", ".jsp", ".sh"]

def detect_file_upload(log_line):
    log = log_line.lower()
    if "content-disposition" in log and "filename=" in log:
        for ext in DANGEROUS_UPLOAD_EXT:
            if ext in log:
                return True
    return False


In [62]:
#final
def analyze_log(log):
    inputs = extract_user_input(log)
    alerts = []

    if detect_sqli(inputs):
        alerts.append("SQL_INJECTION")

    if detect_cmd(inputs):
        alerts.append("COMMAND_INJECTION")

    if detect_path(inputs):
        alerts.append("PATH_TRAVERSAL")

    if detect_xss(inputs):
        alerts.append("XSS")

    if detect_file_upload(log):
        alerts.append("FILE_UPLOAD_ABUSE")

    return alerts if alerts else ["CLEAN"]


In [70]:
raw_logs = [
    "2025-12-30T11:03:19.539529Z,\"('127.0.0.1', 42332)\",GET,http://192.168.122.170/dvwa/vulnerabilities/sqli/?id=1&Submit=Submit",

    "2025-12-30T11:03:19.539529Z,\"('127.0.0.1', 42332)\",GET,http://192.168.122.170/dvwa/vulnerabilities/sqli/?id=%27+OR+1%3D1+--&Submit=Submit",

    "2025-12-30T11:06:09.572621Z,\"('127.0.0.1', 58512)\",GET,http://192.168.122.170/dvwa/vulnerabilities/fi/?page=../index.php",

    "2025-12-30T11:07:58.211196Z,\"('127.0.0.1', 38696)\",POST,http://192.168.122.170/dvwa/vulnerabilities/exec/,ip=%3Bls&submit=submit",

    "filename=\"file.py\" content-disposition: form-data; name=\"uploaded\" content-type=text/python",
    
    "2025-12-30T11:41:16.212493Z,\"('127.0.0.1', 37666)\",GET,http://192.168.122.170/dvwa/vulnerabilities/xss_r/?name=JohnDoe",
    
    "2025-12-30T11:41:16.212493Z,\"('127.0.0.1', 37666)\",GET,http://192.168.122.170/dvwa/vulnerabilities/xss_r/?name=%3Cscript%3Ealert(\"Hacked\")%3C%2Fscript%3E",
    
    "2025-12-30T11:07:58.211196Z,\"('127.0.0.1', 38696)\",POST,http://192.168.122.170/dvwa/vulnerabilities/exec/,ip=%3Bls&submit=submit",
    
    "2025-12-30T11:07:58.211196Z,\"('127.0.0.1', 38696)\",POST,http://192.168.122.170/dvwa/vulnerabilities/exec/,ip=192.168.1.1&submit=submit",
    
    "2025-12-30T11:06:09.572621Z,\"('127.0.0.1', 58512)\",GET,http://192.168.122.170/dvwa/vulnerabilities/fi/?page=include.php",
    
    "2025-12-30T11:03:19.539529Z,\"('127.0.0.1', 42332)\",GET,http://192.168.122.170/dvwa/vulnerabilities/sqli/?id=%27+OR+1%3D1+--&Submit=Submit"






]

for i, log in enumerate(raw_logs, 1):
    print(f"\nRequest {i}")
    print("Detected:", analyze_log(log))



Request 1
Detected: ['CLEAN']

Request 2
Detected: ['SQL_INJECTION']

Request 3
Detected: ['PATH_TRAVERSAL']

Request 4
Detected: ['COMMAND_INJECTION']

Request 5
Detected: ['FILE_UPLOAD_ABUSE']

Request 6
Detected: ['CLEAN']

Request 7
Detected: ['XSS']

Request 8
Detected: ['COMMAND_INJECTION']

Request 9
Detected: ['CLEAN']

Request 10
Detected: ['CLEAN']

Request 11
Detected: ['SQL_INJECTION']
